<a href="https://colab.research.google.com/github/leodatagit/fb_ads_marketing_analytics/blob/main/FBads_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#What we want to discover with this data?
1. Wich campaign was more effective?
  - Wich one brought more conversions?
  - Wich one brought more conversion with less spent?
  - Wich ad had more conversions? 

2. Click rate by USD spent by campaing

3. What's the profile of converted clients?
  - Wich public converted more?

4. How to optimize these campaigns?


#Step by step procedure

1. Importing libraries
2. Exploring the datas
3. Identifying problems in the dataset
4. Fixing our dataset
5. Transforming some variables
6. EDA (Exploratory Data Analysis)
7. Interpret the data and provide suggestions

#1.0 Importing the libraries and data

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
import datetime as dt
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go

##1.1 Mounting the drive, importing the data and creating the dataset

In [2]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/da_projects/datasets/data_fb.csv')

#2.0 Exploring the data

In [4]:
df.head()

,ad_id,reporting_start,reporting_end,campaign_id,fb_campaign_id,age,gender,interest1,interest2,interest3,impressions,clicks,spent,total_conversion,approved_conversion
0,708746,17/08/2017,17/08/2017,916,103916,30-34,M,15,17,17,7350.0,1,1.43,2.0,1.0
1,708749,17/08/2017,17/08/2017,916,103917,30-34,M,16,19,21,17861.0,2,1.82,2.0,0.0
2,708771,17/08/2017,17/08/2017,916,103920,30-34,M,20,25,22,693.0,0,0.00,1.0,0.0
3,708815,30/08/2017,30/08/2017,916,103928,30-34,M,28,32,32,4259.0,1,1.25,1.0,0.0
4,708818,17/08/2017,17/08/2017,916,103928,30-34,M,28,33,32,4133.0,1,1.29,1.0,1.0


In [5]:
df.tail()
#Looks like we have some issues in our dataset

,ad_id,reporting_start,reporting_end,campaign_id,fb_campaign_id,age,gender,interest1,interest2,interest3,impressions,clicks,spent,total_conversion,approved_conversion
1138,1314410,19/08/2017,19/08/2017,45-49,F,109,111,114,1129773,252,358.189997,13,2.0,NaN,NaN
1139,1314411,19/08/2017,19/08/2017,45-49,F,110,111,116,637549,120,173.880003,3,0.0,NaN,NaN
1140,1314412,19/08/2017,19/08/2017,45-49,F,111,113,117,151531,28,40.289999,2,0.0,NaN,NaN
1141,1314414,17/08/2017,17/08/2017,45-49,F,113,114,117,790253,135,198.710001,8,2.0,NaN,NaN
1142,1314415,17/08/2017,17/08/2017,45-49,F,114,116,118,513161,114,165.609999,5,2.0,NaN,NaN


In [6]:
#listing the dataset size (lines, columns)
df.shape

(1143, 15)

In [7]:
#data types
df.dtypes

#We gonna need to transform some datas: 
#(reporting_start, reporting_end) to datetime; 
#(campaign_id, fb_campaign_id) to int; 
#(age) to int; 
#(impressions) to int     

ad_id                    int64
reporting_start         object
reporting_end           object
campaign_id             object
fb_campaign_id          object
age                     object
gender                  object
interest1                int64
interest2                int64
interest3                int64
impressions            float64
clicks                   int64
spent                  float64
total_conversion       float64
approved_conversion    float64
dtype: object

In [8]:
df.info()
#There're nulls on total_conversion and approved_conversion

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ad_id                1143 non-null   int64  
 1   reporting_start      1143 non-null   object 
 2   reporting_end        1143 non-null   object 
 3   campaign_id          1143 non-null   object 
 4   fb_campaign_id       1143 non-null   object 
 5   age                  1143 non-null   object 
 6   gender               1143 non-null   object 
 7   interest1            1143 non-null   int64  
 8   interest2            1143 non-null   int64  
 9   interest3            1143 non-null   int64  
 10  impressions          1143 non-null   float64
 11  clicks               1143 non-null   int64  
 12  spent                1143 non-null   float64
 13  total_conversion     761 non-null    float64
 14  approved_conversion  761 non-null    float64
dtypes: float64(4), int64(5), object(6)
mem

In [9]:
df.duplicated().sum()
#No data duplicated

0

In [10]:
df.nunique()
#verifying the ammount of unique data by variable
#there're 7 differents campaign with 490 different fb_campaign
#gender is strange, should have "only 2 genders"

ad_id                  1143
reporting_start          14
reporting_end            14
campaign_id               7
fb_campaign_id          490
age                      44
gender                   63
interest1                64
interest2               431
interest3               168
impressions            1126
clicks                   97
spent                   509
total_conversion         24
approved_conversion      15
dtype: int64

In [11]:
df['gender']
#Should discover how many 'gender' is compromised

0         M
1         M
2         M
3         M
4         M
       ... 
1138    111
1139    111
1140    113
1141    114
1142    116
Name: gender, Length: 1143, dtype: object

##2.1 Discoverying how divergent data are in some variables
  - age
  - gender
  - campaign_id
  - fb_campaign_id


In [12]:
df['age']

0       30-34
1       30-34
2       30-34
3       30-34
4       30-34
        ...  
1138      109
1139      110
1140      111
1141      113
1142      114
Name: age, Length: 1143, dtype: object

In [13]:
#Listing all the disponible 'age'we have on our dataset
age_datas = []
for i in df['age']:
  if i not in age_datas:
    age_datas.append(i)

age_datas

#It's possible to observe that there are 2 different patterns. 1 is a range of age and the other is a integer number
#And it's known that FB only allow >18 years old, as well it's strange some ages >90~100

['30-34',
 '35-39',
 '40-44',
 '45-49',
 '10',
 '15',
 '16',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '31',
 '32',
 '36',
 '63',
 '64',
 '65',
 '2',
 '66',
 '30',
 '7',
 '100',
 '101',
 '102',
 '103',
 '105',
 '107',
 '110',
 '111',
 '112',
 '113',
 '108',
 '109',
 '114',
 '104',
 '106']

In [14]:
gender_datas = []
for i in df['gender']:
  if i not in gender_datas:
    gender_datas.append(i)

gender_datas

['M',
 'F',
 '14',
 '21',
 '19',
 '17',
 '20',
 '22',
 '24',
 '25',
 '23',
 '26',
 '27',
 '28',
 '29',
 '30',
 '33',
 '31',
 '32',
 '34',
 '35',
 '36',
 '37',
 '38',
 '68',
 '64',
 '65',
 '69',
 '67',
 '5',
 '71',
 '13',
 '18',
 '66',
 '8',
 '6',
 '10',
 '72',
 '15',
 '16',
 '70',
 '4',
 '9',
 '12',
 '41',
 '11',
 '106',
 '104',
 '107',
 '108',
 '112',
 '117',
 '116',
 '105',
 '110',
 '113',
 '114',
 '109',
 '115',
 '102',
 '103',
 '111',
 '118']

In [15]:
campaignid_datas = []
for i in df['campaign_id']:
  if i not in campaignid_datas:
    campaignid_datas.append(i)

campaignid_datas

#Here we can see that there is age patterns in the campaign_id columns

['916', '936', '1178', '45-49', '30-34', '35-39', '40-44']

In [16]:
fb_campaign_id_datas = []

for i in df['fb_campaign_id']:
  if i not in fb_campaign_id_datas:
    fb_campaign_id_datas.append(i)

fb_campaign_id_datas

#The M and F appeared at the fb_campaign_id

['103916',
 '103917',
 '103920',
 '103928',
 '103929',
 '103940',
 '103941',
 '103951',
 '103952',
 '103955',
 '103962',
 '103965',
 '103968',
 '103976',
 '103978',
 '103979',
 '103988',
 '103989',
 '104012',
 '104013',
 '104034',
 '104049',
 '104061',
 '104085',
 '104108',
 '104109',
 '104133',
 '104140',
 '104185',
 '104205',
 '104220',
 '104228',
 '104229',
 '104230',
 '104239',
 '104252',
 '104265',
 '104270',
 '104272',
 '104285',
 '104287',
 '104328',
 '104396',
 '104419',
 '104423',
 '104438',
 '104467',
 '108654',
 '108655',
 '108660',
 '108664',
 '108665',
 '108668',
 '108672',
 '108678',
 '108680',
 '108683',
 '108686',
 '108689',
 '108690',
 '108691',
 '108692',
 '108716',
 '108720',
 '108729',
 '108730',
 '108740',
 '108742',
 '108750',
 '108752',
 '108753',
 '108761',
 '108762',
 '108764',
 '108766',
 '108770',
 '108774',
 '108776',
 '108781',
 '108786',
 '108788',
 '108791',
 '108792',
 '108793',
 '108794',
 '108797',
 '108804',
 '108809',
 '108810',
 '108811',
 '108817',

In [17]:
#Looking at the dataset, we could understand that some campaign_id and fb_campaign_id was missed, and it's filled with information from the other variable
#We have to adjust the dataset, moving some informations from one column to another

##2.2 Discoverying where the data started to get compromised

In [18]:
for i, v in enumerate(df['gender']):
  if v != 'M' and v != 'F':
    print(i)
#The data start to be compromised in the index 761
#Let's see if there're some similarities with another variables 

761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999
1000
1001
1002
1003
1004
1005
1006
1007
1008

In [19]:
for i, v in enumerate(df['campaign_id']):
  print(i, v)
#As suspected, the data start to lose the pattern at the same ID of the other variables
#Discovered that some columns is filling the information of another columns

0 916
1 916
2 916
3 916
4 916
5 916
6 916
7 916
8 916
9 916
10 916
11 916
12 916
13 916
14 916
15 916
16 916
17 916
18 916
19 916
20 916
21 916
22 916
23 916
24 916
25 916
26 916
27 916
28 916
29 916
30 916
31 916
32 916
33 916
34 916
35 916
36 916
37 916
38 916
39 916
40 916
41 916
42 916
43 916
44 916
45 916
46 916
47 916
48 916
49 916
50 916
51 916
52 916
53 916
54 936
55 936
56 936
57 936
58 936
59 936
60 936
61 936
62 936
63 936
64 936
65 936
66 936
67 936
68 936
69 936
70 936
71 936
72 936
73 936
74 936
75 936
76 936
77 936
78 936
79 936
80 936
81 936
82 936
83 936
84 936
85 936
86 936
87 936
88 936
89 936
90 936
91 936
92 936
93 936
94 936
95 936
96 936
97 936
98 936
99 936
100 936
101 936
102 936
103 936
104 936
105 936
106 936
107 936
108 936
109 936
110 936
111 936
112 936
113 936
114 936
115 936
116 936
117 936
118 936
119 936
120 936
121 936
122 936
123 936
124 936
125 936
126 936
127 936
128 936
129 936
130 936
131 936
132 936
133 936
134 936
135 936
136 936
137 936
138 93

#3.0 Data Preparation

In [20]:
#Creating a df copy to manipulate our data
df2 = df.copy()

##3.1 Adjusting the dataset

###3.1.1 Spliting the dataset from the line with error and fixing this part

In [21]:
#Spliting the dataset from the line with error
df2_fixed = df2.loc[761:]

In [22]:
df2_fixed.head()

,ad_id,reporting_start,reporting_end,campaign_id,fb_campaign_id,age,gender,interest1,interest2,interest3,impressions,clicks,spent,total_conversion,approved_conversion
761,1121594,26/08/2017,26/08/2017,45-49,M,10,14,14,426500,72,128.279999,4,1.0,NaN,NaN
762,1121597,30/08/2017,30/08/2017,45-49,M,15,21,19,54237,7,10.780000,2,1.0,NaN,NaN
763,1121598,30/08/2017,30/08/2017,45-49,M,15,19,18,506916,89,133.699999,2,2.0,NaN,NaN
764,1121599,30/08/2017,30/08/2017,45-49,M,15,17,18,250960,42,64.880000,2,0.0,NaN,NaN
765,1121601,30/08/2017,30/08/2017,45-49,M,16,20,18,2286228,353,603.380002,16,7.0,NaN,NaN


In [23]:
df2_fixed.columns

Index(['ad_id', 'reporting_start', 'reporting_end', 'campaign_id',
       'fb_campaign_id', 'age', 'gender', 'interest1', 'interest2',
       'interest3', 'impressions', 'clicks', 'spent', 'total_conversion',
       'approved_conversion'],
      dtype='object')

In [24]:
#We could observe that the missed information in "campaign_id" and "fb_campaign_id" was filled with the information from the others variables. So we need to put all the information
#from the index 761 and further to 2 variables at right, jumping the "campaign_id" and "fb_campaign_id" (letting it empty)
for i, v in reversed(list(enumerate(df2_fixed.columns))):  
  if v == 'fb_campaign_id':
    break
  else:
    df2_fixed[v] = df2_fixed[df2_fixed.columns[i-2]]

df2_fixed['fb_campaign_id'] = 0
df2_fixed['campaign_id'] = 0

<ipython-input-24-646a4af3cf85>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_fixed[v] = df2_fixed[df2_fixed.columns[i-2]]
<ipython-input-24-646a4af3cf85>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_fixed['fb_campaign_id'] = 0
<ipython-input-24-646a4af3cf85>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [25]:
df2_fixed.head()

,ad_id,reporting_start,reporting_end,campaign_id,fb_campaign_id,age,gender,interest1,interest2,interest3,impressions,clicks,spent,total_conversion,approved_conversion
761,1121594,26/08/2017,26/08/2017,0,0,45-49,M,10,14,14,426500,72,128.279999,4,1.0
762,1121597,30/08/2017,30/08/2017,0,0,45-49,M,15,21,19,54237,7,10.780000,2,1.0
763,1121598,30/08/2017,30/08/2017,0,0,45-49,M,15,19,18,506916,89,133.699999,2,2.0
764,1121599,30/08/2017,30/08/2017,0,0,45-49,M,15,17,18,250960,42,64.880000,2,0.0
765,1121601,30/08/2017,30/08/2017,0,0,45-49,M,16,20,18,2286228,353,603.380002,16,7.0


In [26]:
df2_fixed.shape

(382, 15)

In [27]:
#We are going to analyze the data through the campaign_id perspective, ignoring the fb_campaign_id
#Since we don't know if all these data from df2_fixed is from one or more campaigns, we will presume that is just one campaign
#This campaign will have the number 1500 (aleatory)
df2_fixed['campaign_id'] = 1500
df2_fixed.head()

<ipython-input-27-40fb0205e3ff>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_fixed['campaign_id'] = 1500


,ad_id,reporting_start,reporting_end,campaign_id,fb_campaign_id,age,gender,interest1,interest2,interest3,impressions,clicks,spent,total_conversion,approved_conversion
761,1121594,26/08/2017,26/08/2017,1500,0,45-49,M,10,14,14,426500,72,128.279999,4,1.0
762,1121597,30/08/2017,30/08/2017,1500,0,45-49,M,15,21,19,54237,7,10.780000,2,1.0
763,1121598,30/08/2017,30/08/2017,1500,0,45-49,M,15,19,18,506916,89,133.699999,2,2.0
764,1121599,30/08/2017,30/08/2017,1500,0,45-49,M,15,17,18,250960,42,64.880000,2,0.0
765,1121601,30/08/2017,30/08/2017,1500,0,45-49,M,16,20,18,2286228,353,603.380002,16,7.0


###3.1.2 Removing the part with error in our dataset

In [28]:
#Removing the part with error
df2.drop(df2.index[761:], axis=0, inplace=True)
df2.tail()

,ad_id,reporting_start,reporting_end,campaign_id,fb_campaign_id,age,gender,interest1,interest2,interest3,impressions,clicks,spent,total_conversion,approved_conversion
756,1121585,20/08/2017,20/08/2017,1178,144621,40-44,M,66,72,68,9773.0,1,1.460000,1.0,0.0
757,1121589,20/08/2017,20/08/2017,1178,144622,45-49,M,10,16,11,464036.0,77,123.550000,3.0,1.0
758,1121590,20/08/2017,20/08/2017,1178,144622,45-49,M,10,16,15,478480.0,75,135.750001,3.0,1.0
759,1121592,20/08/2017,20/08/2017,1178,144622,45-49,M,10,14,11,428812.0,66,116.880000,4.0,2.0
760,1121593,26/08/2017,26/08/2017,1178,144622,45-49,M,10,16,16,1177535.0,221,365.660001,15.0,3.0


In [29]:
df2_fixed.shape

(382, 15)

In [30]:
df2.shape

(761, 15)

In [31]:
#Verifying if we didn't lost any row
df2_fixed.shape[0] + df2.shape[0]

1143

###3.1.3 Adding the 2 datasets in one

In [32]:
df_union = pd.concat([df2, df2_fixed])
df_union

,ad_id,reporting_start,reporting_end,campaign_id,fb_campaign_id,age,gender,interest1,interest2,interest3,impressions,clicks,spent,total_conversion,approved_conversion
0,708746,17/08/2017,17/08/2017,916,103916,30-34,M,15,17,17,7350.0,1,1.430000,2.0,1.0
1,708749,17/08/2017,17/08/2017,916,103917,30-34,M,16,19,21,17861.0,2,1.820000,2.0,0.0
2,708771,17/08/2017,17/08/2017,916,103920,30-34,M,20,25,22,693.0,0,0.000000,1.0,0.0
3,708815,30/08/2017,30/08/2017,916,103928,30-34,M,28,32,32,4259.0,1,1.250000,1.0,0.0
4,708818,17/08/2017,17/08/2017,916,103928,30-34,M,28,33,32,4133.0,1,1.290000,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,1314410,19/08/2017,19/08/2017,1500,0,45-49,F,109,111,114,1129773.0,252,358.189997,13.0,2.0
1139,1314411,19/08/2017,19/08/2017,1500,0,45-49,F,110,111,116,637549.0,120,173.880003,3.0,0.0
1140,1314412,19/08/2017,19/08/2017,1500,0,45-49,F,111,113,117,151531.0,28,40.289999,2.0,0.0
1141,1314414,17/08/2017,17/08/2017,1500,0,45-49,F,113,114,117,790253.0,135,198.710001,8.0,2.0


In [33]:
df_union.shape

(1143, 15)

In [34]:
#After analyzing some patterns in the ad_id, I could observe that the pattern number of ad_id changes when the campaign_id change, so I could understand that in reality, the campaign_id
#number 1178 goes until the Index 1048, so I just replaced the campaign_id to 1178 until the index 1048 and let the rest of campaign_id with a number of 1500 as we presumed earlier.
df_union.loc[761:1048, ['campaign_id']] = 1178
df_union.loc[761:1048]

,ad_id,reporting_start,reporting_end,campaign_id,fb_campaign_id,age,gender,interest1,interest2,interest3,impressions,clicks,spent,total_conversion,approved_conversion
761,1121594,26/08/2017,26/08/2017,1178,0,45-49,M,10,14,14,426500.0,72,128.279999,4.0,1.0
762,1121597,30/08/2017,30/08/2017,1178,0,45-49,M,15,21,19,54237.0,7,10.780000,2.0,1.0
763,1121598,30/08/2017,30/08/2017,1178,0,45-49,M,15,19,18,506916.0,89,133.699999,2.0,2.0
764,1121599,30/08/2017,30/08/2017,1178,0,45-49,M,15,17,18,250960.0,42,64.880000,2.0,0.0
765,1121601,30/08/2017,30/08/2017,1178,0,45-49,M,16,20,18,2286228.0,353,603.380002,16.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,1122311,27/08/2017,27/08/2017,1178,0,45-49,F,64,69,68,179894.0,43,66.839999,2.0,0.0
1045,1122312,27/08/2017,27/08/2017,1178,0,45-49,F,64,70,66,479882.0,131,178.670001,6.0,0.0
1046,1122313,26/08/2017,26/08/2017,1178,0,45-49,F,64,70,69,358261.0,91,130.360001,1.0,0.0
1047,1122316,26/08/2017,26/08/2017,1178,0,45-49,F,65,67,67,346688.0,88,114.860000,2.0,0.0


##3.2 Transforming the data types

In [35]:
df_union.info()
#We will need to transform some datas: 
#(reporting_start, reporting_end) to datetime; 
#(campaign_id, fb_campaign_id) to int; 
#(interest1) to int;           
#(interest2) to int;
#(impressions) to int
#(total_conversion) to int
#(approved_conversion) to int

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1143 entries, 0 to 1142
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ad_id                1143 non-null   int64  
 1   reporting_start      1143 non-null   object 
 2   reporting_end        1143 non-null   object 
 3   campaign_id          1143 non-null   object 
 4   fb_campaign_id       1143 non-null   object 
 5   age                  1143 non-null   object 
 6   gender               1143 non-null   object 
 7   interest1            1143 non-null   object 
 8   interest2            1143 non-null   object 
 9   interest3            1143 non-null   int64  
 10  impressions          1143 non-null   float64
 11  clicks               1143 non-null   int64  
 12  spent                1143 non-null   float64
 13  total_conversion     1143 non-null   float64
 14  approved_conversion  1143 non-null   float64
dtypes: float64(4), int64(3), object(8)
mem

In [36]:
#Transforming datetime
df_union['reporting_start'] = pd.to_datetime(df_union['reporting_start'], format='%d/%m/%Y')
df_union['reporting_end'] = pd.to_datetime(df_union['reporting_end'], format='%d/%m/%Y')

In [37]:
df_union['campaign_id'] = df_union['campaign_id'].astype(int)
df_union['fb_campaign_id'] = df_union['fb_campaign_id'].astype(int)
df_union['interest1'] = df_union['interest1'].astype(int)
df_union['interest2'] = df_union['interest2'].astype(int)
df_union['impressions'] = df_union['impressions'].astype(int)
df_union['total_conversion'] = df_union['total_conversion'].astype(int)
df_union['approved_conversion'] = df_union['approved_conversion'].astype(int)

In [38]:
#Transforming the gender in a boolean
gender = {'M':1, 'F':0}
df_union['gender'] = df_union['gender'].map(gender)

In [39]:
df_union.head()

,ad_id,reporting_start,reporting_end,campaign_id,fb_campaign_id,age,gender,interest1,interest2,interest3,impressions,clicks,spent,total_conversion,approved_conversion
0,708746,2017-08-17,2017-08-17,916,103916,30-34,1,15,17,17,7350,1,1.43,2,1
1,708749,2017-08-17,2017-08-17,916,103917,30-34,1,16,19,21,17861,2,1.82,2,0
2,708771,2017-08-17,2017-08-17,916,103920,30-34,1,20,25,22,693,0,0.00,1,0
3,708815,2017-08-30,2017-08-30,916,103928,30-34,1,28,32,32,4259,1,1.25,1,0
4,708818,2017-08-17,2017-08-17,916,103928,30-34,1,28,33,32,4133,1,1.29,1,1


In [40]:
df_union.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1143 entries, 0 to 1142
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ad_id                1143 non-null   int64         
 1   reporting_start      1143 non-null   datetime64[ns]
 2   reporting_end        1143 non-null   datetime64[ns]
 3   campaign_id          1143 non-null   int64         
 4   fb_campaign_id       1143 non-null   int64         
 5   age                  1143 non-null   object        
 6   gender               1143 non-null   int64         
 7   interest1            1143 non-null   int64         
 8   interest2            1143 non-null   int64         
 9   interest3            1143 non-null   int64         
 10  impressions          1143 non-null   int64         
 11  clicks               1143 non-null   int64         
 12  spent                1143 non-null   float64       
 13  total_conversion     1143 non-nul

In [41]:
df_union.nunique()

ad_id                  1143
reporting_start          14
reporting_end            14
campaign_id               4
fb_campaign_id          489
age                       4
gender                    2
interest1                40
interest2                66
interest3                69
impressions            1130
clicks                  183
spent                   869
total_conversion         32
approved_conversion      16
dtype: int64

##3.2 Enriching the data

Let's create some variables from the data we already have

In [42]:
df_union['CPC'] = df_union['spent'] / df_union['clicks']
df_union['CTR'] = df_union['clicks'] / df_union['impressions']
df_union['CPM_impressions'] = df_union['spent'] / df_union['impressions']
df_union['totconv_rate_click'] = (df_union['total_conversion'] / df_union['clicks'])*100
df_union['spent_totconv'] = df_union['spent'] / df_union['total_conversion']

df_union.head()

,ad_id,reporting_start,reporting_end,campaign_id,fb_campaign_id,age,gender,interest1,interest2,interest3,impressions,clicks,spent,total_conversion,approved_conversion,CPC,CTR,CPM_impressions,totconv_rate_click,spent_totconv
0,708746,2017-08-17,2017-08-17,916,103916,30-34,1,15,17,17,7350,1,1.43,2,1,1.43,0.000136,0.000195,200.0,0.715
1,708749,2017-08-17,2017-08-17,916,103917,30-34,1,16,19,21,17861,2,1.82,2,0,0.91,0.000112,0.000102,100.0,0.910
2,708771,2017-08-17,2017-08-17,916,103920,30-34,1,20,25,22,693,0,0.00,1,0,NaN,0.000000,0.000000,inf,0.000
3,708815,2017-08-30,2017-08-30,916,103928,30-34,1,28,32,32,4259,1,1.25,1,0,1.25,0.000235,0.000293,100.0,1.250
4,708818,2017-08-17,2017-08-17,916,103928,30-34,1,28,33,32,4133,1,1.29,1,1,1.29,0.000242,0.000312,100.0,1.290


In [43]:
#Filling the NaN with 0
df_union['CPC'].fillna(0, inplace=True)
df_union['CTR'].fillna(0, inplace=True)
df_union['CPM_impressions'].fillna(0, inplace=True)
df_union['totconv_rate_click'].fillna(0, inplace=True)
df_union['spent_totconv'].fillna(0, inplace=True)

In [44]:
#Filling the "inf" with 0
df_union.replace([np.inf, -np.inf], 0, inplace=True)

#4.0 EDA (Exploratory Data Analysis)

##Analyzing by the campaign_id spectre

In [45]:
df_union.describe()

,ad_id,campaign_id,fb_campaign_id,gender,interest1,interest2,interest3,impressions,clicks,spent,total_conversion,approved_conversion,CPC,CTR,CPM_impressions,totconv_rate_click,spent_totconv
count,1.143000e+03,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1.143000e+03,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000
mean,9.872611e+05,1093.863517,82501.628171,0.517935,32.766404,36.245844,36.222222,1.867321e+05,33.390201,51.360656,2.855643,0.944007,1.227812,0.000164,0.000239,24.169930,16.016721
std,1.939928e+05,168.789682,59733.441715,0.499897,26.952131,26.937853,26.924679,3.127622e+05,56.892438,86.908418,4.483593,1.737708,0.614889,0.000115,0.000161,36.838128,24.297504
min,7.087460e+05,916.000000,0.000000,0.000000,2.000000,3.000000,3.000000,8.700000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.776325e+05,936.000000,0.000000,0.000000,16.000000,20.000000,20.000000,6.503500e+03,1.000000,1.480000,1.000000,0.000000,1.234500,0.000100,0.000149,2.857143,1.380000
50%,1.121185e+06,1178.000000,110975.000000,1.000000,25.000000,28.000000,28.000000,5.150900e+04,8.000000,12.370000,1.000000,1.000000,1.450000,0.000160,0.000249,9.090909,8.190000
75%,1.121804e+06,1178.000000,123613.500000,1.000000,31.000000,35.000000,35.000000,2.217690e+05,37.500000,60.025000,3.000000,1.000000,1.600000,0.000234,0.000333,25.000000,21.229286
max,1.314415e+06,1500.000000,144622.000000,1.000000,114.000000,118.000000,120.000000,3.052003e+06,421.000000,639.949998,60.000000,21.000000,2.212000,0.001059,0.001504,400.000000,332.989999


In [46]:
df_union.corr()

,ad_id,campaign_id,fb_campaign_id,gender,interest1,interest2,interest3,impressions,clicks,spent,total_conversion,approved_conversion,CPC,CTR,CPM_impressions,totconv_rate_click,spent_totconv
ad_id,1.000000,0.934845,-0.511471,0.058324,0.456483,0.454437,0.455100,0.462308,0.431981,0.433053,0.346539,0.271331,0.481869,-0.047237,0.033330,-0.243767,0.358880
campaign_id,0.934845,1.000000,-0.564261,0.039714,0.629133,0.626616,0.627160,0.439752,0.406381,0.401922,0.348020,0.259434,0.461839,-0.047268,0.019016,-0.226816,0.316281
fb_campaign_id,-0.511471,-0.564261,1.000000,0.409289,-0.367508,-0.366645,-0.364736,-0.305816,-0.393189,-0.354388,-0.161891,-0.090828,-0.189672,-0.144368,-0.146965,0.205742,-0.341742
gender,0.058324,0.039714,0.409289,1.000000,-0.047230,-0.046984,-0.043755,-0.067059,-0.168725,-0.125022,-0.027559,0.025350,0.042440,-0.249175,-0.209005,0.063571,-0.136503
interest1,0.456483,0.629133,-0.367508,-0.047230,1.000000,0.997908,0.998041,0.101973,0.088706,0.070226,0.120270,0.058353,0.097047,-0.051653,-0.048275,-0.067045,0.018602
interest2,0.454437,0.626616,-0.366645,-0.046984,0.997908,1.000000,0.996085,0.101459,0.087988,0.069662,0.119080,0.059108,0.096580,-0.052964,-0.049304,-0.069073,0.018042
interest3,0.455100,0.627160,-0.364736,-0.043755,0.998041,0.996085,1.000000,0.102056,0.088928,0.070409,0.120662,0.058622,0.098751,-0.050716,-0.047221,-0.064748,0.018034
impressions,0.462308,0.439752,-0.305816,-0.067059,0.101973,0.101459,0.102056,1.000000,0.948514,0.970386,0.812838,0.684249,0.329214,0.075914,0.132442,-0.241845,0.430466
clicks,0.431981,0.406381,-0.393189,-0.168725,0.088706,0.087988,0.088928,0.948514,1.000000,0.992906,0.694632,0.559526,0.296515,0.159290,0.202076,-0.257893,0.527938
spent,0.433053,0.401922,-0.354388,-0.125022,0.070226,0.069662,0.070409,0.970386,0.992906,1.000000,0.725379,0.593178,0.315671,0.140952,0.193987,-0.258270,0.516529


In [47]:
#Creating a list of campaign data to automatize process further
campaignid_datas = []
for i in df_union['campaign_id']:
  if i not in campaignid_datas:
    campaignid_datas.append(i)

campaignid_datas

[916, 936, 1178, 1500]

In [48]:
df_union_916 = df_union.loc[(df_union['campaign_id']==916)]
df_union_936 = df_union.loc[(df_union['campaign_id']==936)]
df_union_1178 = df_union.loc[(df_union['campaign_id']==1178)]
df_union_1500 = df_union.loc[(df_union['campaign_id']==1500)]

print(df_union_916.shape)
print(df_union_936.shape)
print(df_union_1178.shape)
print(df_union_1500.shape)

(54, 20)
(464, 20)
(531, 20)
(94, 20)


In [49]:
#Confirming if we haven't lost any row
df_union_916.shape[0] + df_union_936.shape[0] + df_union_1178.shape[0] + df_union_1500.shape[0]

1143

##4.1 Campaign 916

###4.1.1 Social Data

In [50]:
#Create a subplot with 2 rows and 3 columns
fig = sp.make_subplots(rows = 2, cols = 3, subplot_titles = ['Age', 'Gender', 'Reporting end', 'Interest 1', 'Interest 2', 'Interest 3'])

#AGE
fig.add_trace(go.Bar(x = df_union_916.groupby('age').sum().reset_index()['age'], y = df_union_916.groupby('age').sum().reset_index()['total_conversion'], name='Age'), row=1, col=1)

#Gender
fig.add_trace(go.Bar(x = df_union_916.groupby('gender').sum().reset_index()['gender'], y = df_union_916.groupby('gender').sum().reset_index()['total_conversion'], name='Gender'), row=1, col=2)

#Reporting_End
fig.add_trace(go.Bar(x = df_union_916.groupby('reporting_end').sum().reset_index()['reporting_end'], y = df_union_916.groupby('reporting_end').sum().reset_index()['total_conversion'], name='Reporting end'), row=1, col=3)

#Interest1
fig.add_trace(go.Bar(x = df_union_916.groupby('interest1').sum().reset_index()['interest1'], y = df_union_916.groupby('interest1').sum().reset_index()['total_conversion'], name='Interest 1'), row=2, col=1)

#Interest2
fig.add_trace(go.Bar(x = df_union_916.groupby('interest2').sum().reset_index()['interest2'], y = df_union_916.groupby('interest2').sum().reset_index()['total_conversion'], name='Interest 2'), row=2, col=2)

#Interest3
fig.add_trace(go.Bar(x = df_union_916.groupby('interest3').sum().reset_index()['interest3'], y = df_union_916.groupby('interest3').sum().reset_index()['total_conversion'], name='Interest 3'), row=2, col=3)


fig.update_layout(height=600, width=1400, title_text="Campaign 916 (Total conversion)", title_x = 0.5, title_font_size=25)

fig.show()


###4.1.2 Performance

In [51]:
#Create a subplot with 2 rows and 3 columns
fig = sp.make_subplots(rows = 2, cols = 3, subplot_titles = ['Age', 'Gender', 'Reporting end', 'Interest 1', 'Interest 2', 'Interest 3'])

#AGE
fig.add_trace(go.Box(x = df_union_916['age'], y = df_union_916['spent_totconv'], name='Age'), row=1, col=1)

#Gender
fig.add_trace(go.Box(x = df_union_916['gender'], y = df_union_916['spent_totconv'], name='Gender'), row=1, col=2)

#Reporting_End
fig.add_trace(go.Box(x = df_union_916['reporting_end'], y = df_union_916['spent_totconv'], name='Reporting end'), row=1, col=3)

#Interest1
fig.add_trace(go.Box(x = df_union_916['interest1'], y = df_union_916['spent_totconv'], name='Interest 1'), row=2, col=1)

#Interest2
fig.add_trace(go.Box(x = df_union_916['interest2'], y = df_union_916['spent_totconv'], name='Interest 2'), row=2, col=2)

#Interest3
fig.add_trace(go.Box(x = df_union_916['interest3'], y = df_union_916['spent_totconv'], name='Interest 3'), row=2, col=3)


fig.update_layout(height=800, width=1400, title_text="Campaign 916 (Spent / Convertion)", title_x = 0.5, title_font_size=25)

fig.show()


###4.1.3 Convertion Funnel

In [52]:
data = dict(
    number=[df_union_916['impressions'].sum()/1000, df_union_916['clicks'].sum(), df_union_916['total_conversion'].sum(), df_union_916['approved_conversion'].sum()],
    stage=['Impressions / 1000', 'Clicks', 'Total Conversion', 'Approved Conversion'])
fig = px.funnel(data, x='number', y='stage')

fig.update_traces(textinfo='value+percent previous')
fig.show()


##4.2 Campaign 936

Exploring some data through visualization


###4.2.1 Social Data

In [53]:
#Create a subplot with 2 rows and 3 columns
fig = sp.make_subplots(rows = 2, cols = 3, subplot_titles = ['Age', 'Gender', 'Reporting end', 'Interest 1', 'Interest 2', 'Interest 3'])

#AGE
fig.add_trace(go.Bar(x = df_union_936.groupby('age').sum().reset_index()['age'], y = df_union_936.groupby('age').sum().reset_index()['total_conversion'], name='Age'), row=1, col=1)

#Gender
fig.add_trace(go.Bar(x = df_union_936.groupby('gender').sum().reset_index()['gender'], y = df_union_936.groupby('gender').sum().reset_index()['total_conversion'], name='Gender'), row=1, col=2)

#Reporting_End
fig.add_trace(go.Bar(x = df_union_936.groupby('reporting_end').sum().reset_index()['reporting_end'], y = df_union_936.groupby('reporting_end').sum().reset_index()['total_conversion'], name='Reporting end'), row=1, col=3)

#Interest1
fig.add_trace(go.Bar(x = df_union_936.groupby('interest1').sum().reset_index()['interest1'], y = df_union_936.groupby('interest1').sum().reset_index()['total_conversion'], name='Interest 1'), row=2, col=1)

#Interest2
fig.add_trace(go.Bar(x = df_union_936.groupby('interest2').sum().reset_index()['interest2'], y = df_union_936.groupby('interest2').sum().reset_index()['total_conversion'], name='Interest 2'), row=2, col=2)

#Interest3
fig.add_trace(go.Bar(x = df_union_936.groupby('interest3').sum().reset_index()['interest3'], y = df_union_936.groupby('interest3').sum().reset_index()['total_conversion'], name='Interest 3'), row=2, col=3)


fig.update_layout(height=600, width=1400, title_text="Campaign 936 (Total conversion)", title_x = 0.5, title_font_size=25)

fig.show()


###4.2.2 Performance

In [54]:
#Create a subplot with 2 rows and 3 columns
fig = sp.make_subplots(rows = 2, cols = 3, subplot_titles = ['Age', 'Gender', 'Reporting end', 'Interest 1', 'Interest 2', 'Interest 3'])

#AGE
fig.add_trace(go.Box(x = df_union_936['age'], y = df_union_936['spent_totconv'], name='Age'), row=1, col=1)

#Gender
fig.add_trace(go.Box(x = df_union_936['gender'], y = df_union_936['spent_totconv'], name='Gender'), row=1, col=2)

#Reporting_End
fig.add_trace(go.Box(x = df_union_936['reporting_end'], y = df_union_936['spent_totconv'], name='Reporting end'), row=1, col=3)

#Interest1
fig.add_trace(go.Box(x = df_union_936['interest1'], y = df_union_936['spent_totconv'], name='Interest 1'), row=2, col=1)

#Interest2
fig.add_trace(go.Box(x = df_union_936['interest2'], y = df_union_936['spent_totconv'], name='Interest 2'), row=2, col=2)

#Interest3
fig.add_trace(go.Box(x = df_union_936['interest3'], y = df_union_936['spent_totconv'], name='Interest 3'), row=2, col=3)


fig.update_layout(height=800, width=1400, title_text="Campaign 936 (Spent / Convertion)", title_x = 0.5, title_font_size=25)

fig.show()


###4.2.3 Convertion Funnel

In [55]:
data = dict(
    number=[df_union_936['impressions'].sum()/1000, df_union_936['clicks'].sum(), df_union_936['total_conversion'].sum(), df_union_936['approved_conversion'].sum()],
    stage=['Impressions / 1000', 'Clicks', 'Total Conversion', 'Approved Conversion'])
fig = px.funnel(data, x='number', y='stage')

fig.update_traces(textinfo='value+percent previous')

fig.show()

##4.3 Campaign 1178

Exploring some data through visualization

###4.3.1 Social Data

In [56]:
#Create a subplot with 2 rows and 3 columns
fig = sp.make_subplots(rows = 2, cols = 3, subplot_titles = ['Age', 'Gender', 'Reporting end', 'Interest 1', 'Interest 2', 'Interest 3'])

#AGE
fig.add_trace(go.Bar(x = df_union_1178.groupby('age').sum().reset_index()['age'], y = df_union_1178.groupby('age').sum().reset_index()['total_conversion'], name='Age'), row=1, col=1)

#Gender
fig.add_trace(go.Bar(x = df_union_1178.groupby('gender').sum().reset_index()['gender'], y = df_union_1178.groupby('gender').sum().reset_index()['total_conversion'], name='Gender'), row=1, col=2)

#Reporting_End
fig.add_trace(go.Bar(x = df_union_1178.groupby('reporting_end').sum().reset_index()['reporting_end'], y = df_union_1178.groupby('reporting_end').sum().reset_index()['total_conversion'], name='Reporting end'), row=1, col=3)

#Interest1
fig.add_trace(go.Bar(x = df_union_1178.groupby('interest1').sum().reset_index()['interest1'], y = df_union_1178.groupby('interest1').sum().reset_index()['total_conversion'], name='Interest 1'), row=2, col=1)

#Interest2
fig.add_trace(go.Bar(x = df_union_1178.groupby('interest2').sum().reset_index()['interest2'], y = df_union_1178.groupby('interest2').sum().reset_index()['total_conversion'], name='Interest 2'), row=2, col=2)

#Interest3
fig.add_trace(go.Bar(x = df_union_1178.groupby('interest3').sum().reset_index()['interest3'], y = df_union_1178.groupby('interest3').sum().reset_index()['total_conversion'], name='Interest 3'), row=2, col=3)


fig.update_layout(height=600, width=1400, title_text="Campaign 1178 (Total conversion)", title_x = 0.5, title_font_size=25)

fig.show()

###4.3.2 Performance

In [57]:
#Create a subplot with 2 rows and 3 columns
fig = sp.make_subplots(rows = 2, cols = 3, subplot_titles = ['Age', 'Gender', 'Reporting end', 'Interest 1', 'Interest 2', 'Interest 3'])

#AGE
fig.add_trace(go.Box(x = df_union_1178['age'], y = df_union_1178['spent_totconv'], name='Age'), row=1, col=1)

#Gender
fig.add_trace(go.Box(x = df_union_1178['gender'], y = df_union_1178['spent_totconv'], name='Gender'), row=1, col=2)

#Reporting_End
fig.add_trace(go.Box(x = df_union_1178['reporting_end'], y = df_union_1178['spent_totconv'], name='Reporting end'), row=1, col=3)

#Interest1
fig.add_trace(go.Box(x = df_union_1178['interest1'], y = df_union_1178['spent_totconv'], name='Interest 1'), row=2, col=1)

#Interest2
fig.add_trace(go.Box(x = df_union_1178['interest2'], y = df_union_1178['spent_totconv'], name='Interest 2'), row=2, col=2)

#Interest3
fig.add_trace(go.Box(x = df_union_1178['interest3'], y = df_union_1178['spent_totconv'], name='Interest 3'), row=2, col=3)


fig.update_layout(height=800, width=1400, title_text="Campaign 1178 (Spent / Convertion)", title_x = 0.5, title_font_size=25)

fig.show()


###4.3.3 Convertion Funnel

In [58]:
data = dict(
    number=[df_union_1178['impressions'].sum()/1000, df_union_1178['clicks'].sum(), df_union_1178['total_conversion'].sum(), df_union_1178['approved_conversion'].sum()],
    stage=['Impressions / 1000', 'Clicks', 'Total Conversion', 'Approved Conversion'])
fig = px.funnel(data, x='number', y='stage')

fig.update_traces(textinfo='value+percent previous')

fig.show()

##4.4 Campaign 1500

Exploring some data through visualization

###4.4.1 Social Data

In [59]:
#Create a subplot with 2 rows and 3 columns
fig = sp.make_subplots(rows = 2, cols = 3, subplot_titles = ['Age', 'Gender', 'Reporting end', 'Interest 1', 'Interest 2', 'Interest 3'])

#AGE
fig.add_trace(go.Bar(x = df_union_1500.groupby('age').sum().reset_index()['age'], y = df_union_1500.groupby('age').sum().reset_index()['total_conversion'], name='Age'), row=1, col=1)

#Gender
fig.add_trace(go.Bar(x = df_union_1500.groupby('gender').sum().reset_index()['gender'], y = df_union_1500.groupby('gender').sum().reset_index()['total_conversion'], name='Gender'), row=1, col=2)

#Reporting_End
fig.add_trace(go.Bar(x = df_union_1500.groupby('reporting_end').sum().reset_index()['reporting_end'], y = df_union_1500.groupby('reporting_end').sum().reset_index()['total_conversion'], name='Reporting end'), row=1, col=3)

#Interest1
fig.add_trace(go.Bar(x = df_union_1500.groupby('interest1').sum().reset_index()['interest1'], y = df_union_1500.groupby('interest1').sum().reset_index()['total_conversion'], name='Interest 1'), row=2, col=1)

#Interest2
fig.add_trace(go.Bar(x = df_union_1500.groupby('interest2').sum().reset_index()['interest2'], y = df_union_1500.groupby('interest2').sum().reset_index()['total_conversion'], name='Interest 2'), row=2, col=2)

#Interest3
fig.add_trace(go.Bar(x = df_union_1500.groupby('interest3').sum().reset_index()['interest3'], y = df_union_1500.groupby('interest3').sum().reset_index()['total_conversion'], name='Interest 3'), row=2, col=3)


fig.update_layout(height=600, width=1400, title_text="Campaign 1500 (Total conversion)", title_x = 0.5, title_font_size=25)

fig.show()

###4.4.2 Performance

In [60]:
fig = sp.make_subplots(rows = 2, cols = 3, subplot_titles = ['Age', 'Gender', 'Reporting end', 'Interest 1', 'Interest 2', 'Interest 3'])

#AGE
fig.add_trace(go.Box(x = df_union_1500['age'], y = df_union_1500['spent_totconv'], name='Age'), row=1, col=1)

#Gender
fig.add_trace(go.Box(x = df_union_1500['gender'], y = df_union_1500['spent_totconv'], name='Gender'), row=1, col=2)

#Reporting_End
fig.add_trace(go.Box(x = df_union_1500['reporting_end'], y = df_union_1500['spent_totconv'], name='Reporting end'), row=1, col=3)

#Interest1
fig.add_trace(go.Box(x = df_union_1500['interest1'], y = df_union_1500['spent_totconv'], name='Interest 1'), row=2, col=1)

#Interest2
fig.add_trace(go.Box(x = df_union_1500['interest2'], y = df_union_1500['spent_totconv'], name='Interest 2'), row=2, col=2)

#Interest3
fig.add_trace(go.Box(x = df_union_1500['interest3'], y = df_union_1500['spent_totconv'], name='Interest 3'), row=2, col=3)


fig.update_layout(height=800, width=1400, title_text="Campaign 1500 (Spent / Convertion)", title_x = 0.5, title_font_size=25)

fig.show()


###4.4.3 Convertion Funnel

In [61]:
data = dict(
    number=[df_union_1500['impressions'].sum()/1000, df_union_1500['clicks'].sum(), df_union_1500['total_conversion'].sum(), df_union_1500['approved_conversion'].sum()],
    stage=['Impressions / 1000', 'Clicks', 'Total Conversion', 'Approved Conversion'])
fig = px.funnel(data, x='number', y='stage')

fig.update_traces(textinfo='value+percent previous')

fig.show()

#5.0 Answering some question

##5.1 Wich campaign was more effective?
  - Wich campaign brought more conversions?
  - What was the average cost per conversion in general, and per campaign?
  - What was the average cost per gender / age / interest?
  - Wich campaign brought more conversion with less spent?
  - Wich ad had more conversions? (with a cost below the average)


In [62]:
#Total conversion per campaign_id
for i in campaignid_datas:
  print(f'Campaign', i, '=', df_union[df_union['campaign_id']==i]['total_conversion'].sum())

Campaign 916 = 58
Campaign 936 = 537
Campaign 1178 = 2137
Campaign 1500 = 532


In [63]:
fig = px.bar(df_union.groupby('campaign_id').sum().reset_index(), x='campaign_id', y='total_conversion', height=400, width=500, title='Total conversion')
fig.update_xaxes(type='category')
fig.update_layout(title_x = 0.5, title_font_size=25)
fig.show()

In [64]:
#Average Cost per conversion
avg_cost_conversion = df_union['spent'].sum()/df_union['total_conversion'].sum()
print(f'Average cost per conversion = ', avg_cost_conversion)

Average cost per conversion =  17.985670943077512


In [65]:
#Cost per conversion per campaign
for i in campaignid_datas:
  print(f'Campaign', i, 'cost per conversion =', df_union[df_union['campaign_id']==i]['spent'].sum() / df_union[df_union['campaign_id']==i]['total_conversion'].sum())

Campaign 916 cost per conversion = 2.58120690787931
Campaign 936 cost per conversion = 5.388026068778399
Campaign 1178 cost per conversion = 22.36538604157838
Campaign 1500 cost per conversion = 14.788195465716166


In [66]:
fig = px.bar(df_union.groupby('campaign_id').sum().reset_index(), 
             x='campaign_id', 
             y=df_union.groupby('campaign_id').sum().reset_index()['spent']/df_union.groupby('campaign_id').sum().reset_index()['total_conversion'], 
             height=400, width=500, title='Cost per conversion')
fig.update_xaxes(type='category')
fig.update_layout(title_x = 0.5, title_font_size=25)
fig.show()

In [67]:
#Approved Conversion per Total Conversion rate
for i in campaignid_datas:
  print(f'Campaign', i, 'approved conversion / total conversion rate =', df_union[df_union['campaign_id']==i]['approved_conversion'].sum() / df_union[df_union['campaign_id']==i]['total_conversion'].sum())

Campaign 916 approved conversion / total conversion rate = 0.41379310344827586
Campaign 936 approved conversion / total conversion rate = 0.3407821229050279
Campaign 1178 approved conversion / total conversion rate = 0.3401965372016846
Campaign 1500 approved conversion / total conversion rate = 0.2725563909774436


In [68]:
#The 10 top conversion ads
df_union.sort_values(by='total_conversion', ascending=False).head(10)[['ad_id', 'campaign_id', 'total_conversion', 'spent_totconv']]

,ad_id,campaign_id,total_conversion,spent_totconv
525,1121100,1178,60,10.665833
528,1121104,1178,40,9.003750
860,1121814,1178,38,16.113158
1116,1314387,1500,31,7.199677
574,1121196,1178,31,11.566129
1127,1314398,1500,30,10.035000
518,1121091,1178,28,9.073214
531,1121108,1178,26,6.303846
827,1121746,1178,26,10.309615
524,1121098,1178,24,9.865417


In [69]:
#Top 10 ads by conversion and cost per conversion below the avg cost per conversion of all campaigns
df_union[df_union['spent_totconv']<avg_cost_conversion].sort_values(by='total_conversion', ascending=False).head(10)[['ad_id', 'campaign_id', 'total_conversion', 'spent', 'spent_totconv']]

,ad_id,campaign_id,total_conversion,spent,spent_totconv
525,1121100,1178,60,639.949998,10.665833
528,1121104,1178,40,360.150001,9.003750
860,1121814,1178,38,612.300003,16.113158
1116,1314387,1500,31,223.189995,7.199677
574,1121196,1178,31,358.550003,11.566129
1127,1314398,1500,30,301.049999,10.035000
518,1121091,1178,28,254.049996,9.073214
531,1121108,1178,26,163.899997,6.303846
827,1121746,1178,26,268.050002,10.309615
524,1121098,1178,24,236.769999,9.865417


In [70]:
#Top 10 ads by conversion and cost per conversion below 8.00 per conversion of all campaigns
df_union[df_union['spent_totconv']<8].sort_values(by='total_conversion', ascending=False).head(10)[['ad_id', 'campaign_id', 'total_conversion', 'spent', 'spent_totconv']]

,ad_id,campaign_id,total_conversion,spent,spent_totconv
1116,1314387,1500,31,223.189995,7.199677
531,1121108,1178,26,163.899997,6.303846
1101,1314367,1500,23,163.800000,7.121739
579,1121206,1178,22,35.310000,1.605000
1094,1314360,1500,21,110.780002,5.275238
1097,1314363,1500,20,119.640002,5.982000
1054,1314306,1500,11,46.669999,4.242727
1053,1314303,1500,10,61.009999,6.101000
584,1121216,1178,8,48.610000,6.076250
580,1121207,1178,8,63.320001,7.915000


In [71]:
#The 10 top expensive spent/conversion ads
df_union.sort_values(by='spent_totconv', ascending=False).head(10)[['ad_id', 'campaign_id', 'total_conversion', 'spent', 'spent_totconv']]

,ad_id,campaign_id,total_conversion,spent,spent_totconv
1003,1122209,1178,1,332.989999,332.989999
999,1122202,1178,1,295.549996,295.549996
207,776325,936,1,180.220001,180.220001
1015,1122233,1178,1,145.819997,145.819997
1046,1122313,1178,1,130.360001,130.360001
1016,1122240,1178,1,125.270001,125.270001
1126,1314397,1500,1,120.899998,120.899998
812,1121691,1178,3,352.449999,117.483333
996,1122197,1178,2,234.939999,117.470000
1005,1122211,1178,1,115.660001,115.660001


In [73]:
#Exporting the dataset df_union
df_union.to_csv('/content/gdrive/MyDrive/Colab Notebooks/da_projects/fb_ads_campaigns/df_union.csv')

#6.0 How to optimize the campaigns to reduce the cost per convertion?

We can approach the optimization of these campaigns by 3 aspects:

1. Social
2. Performance
3. Conversion funnel

While the analyzing process, some patterns became evident and got our attention.

I'll pass through each campaign to depict better some suggestion of optimization

##6.1 Campaign 916

####Social
By the social aspects we can clearly see that this campaign had much more results at the segment ages 30-34 and 35-39, corresponding 75% of conversion, and Male. Also the interests that had more conversion was between 15-38, with a spikes in 16, 19 and 20
Between the days 20 to 24, there were no conversions, could be something to investigate better, if there weren't announces or something else that could have happened.

####Performance
Looking at the cost per conversion it's crystal clear that the segments of age 40-44 and 45-49 was much more expensive to convert (it had a little ammount of data). Women public was also more expensive.

####Conversion Funnel
The conversion funnel shows off a good rate of conversion when the public clicked at the link. We could filter the best total_conversion/clicks rate ad_id and try to model these ads patterns in new ads
Also if we can optimize the CTR, even just a little, we can have a great increase of conversion if we can keep the total_conversion rate the same.

Notes: The cost of conversion of this campaign was the smallest, so it'd be very interesting to increase the budget on this campaign focusing more in Male between 30-39 and interests between 15-38.



##6.2 Campaign 936
####Social
Much more results at the segment ages 30-34 and 45-49, corresponding 66% of conversion. As M and F had almost the same conversion in quantity. Also the interests that had more conversion was between 10 -35, with a spikes in 10, 16, 17, 19, 20 and 32
From the day 24 it started to get few conversion, it'd be interesting to analyze if the budget still the same or it droped.

####Performance
The segments of age 40-44 and mainly 45-49 was much more expensive to convert. The segment 30-34 was eminently cheaper. Women public was also more expensive, costing 3x(median) more than man.

####Conversion Funnel
The CTR was the pratically the same as the campaign 916, but the total_conversion/clicks was almost the half. It'd be interesting to understand the pattern of conversion from the campaign_916 and use in this campaign

**Notes:** The cost per conversion of this campaign was also low, so it'd be interesting to increase its budget, focusing in the age segment 30-39. Even the women being more expensive than man, I think it could be interesting to keep announcing for them, as we saw that the conversion similarly to man.


##6.3 Campaign 1178
####Social
Much more results at the segment ages 30-34, corresponding almost 50% of conversion. Woman had a few more conversions than man, representing 57% of total conversion.
Also the interests that had more conversion was between 10-35, with a spikes in10, 15, 16, 20, 27, 30, 31, 34
The conversions started to grow up from the day 23 of august. It'd be interesting to see if it had a surpluss on the budget.

####Performance
The 45-49 segment was too way expensive, the median was almost 3x the 30-34 segment. The median cost by F was almost 45% more expensive, and with some outliers in F.


####Conversion Funnel
The CTR had a noticeable worsen in comparison, also the total_conversion rate had a big drop. This fenomenum should be investigate.

**Notes:** The cost per conversion the bigger one, so to lower it, would be interesting to focus more in the 30-39 public. I'd keep the announcing to F public and use the interests that had more results.
Maybe trying to model the ads of campaign_916 and 936 could improve the conversion rates and get a big surpluss in the results.



##6.4 Campaign 1500
####Social
The distribution of convertion by rate was the most similar between all campaigns. The M public had a massive convertion if compare with F public.
Also we can see that the majority of convertion was untill day 23 aug.
The interests that had more conversion was between 100-116, with spikes in 101, 105, 107, 112 and 113.

####Performance
The segments 40-44 and 45-49 was the most expensive as we saw in others campaign, but in this case it had a good conversion. The F public was around 50% more expensive than M (median)

####Conversion Funnel
As saw in the 1178 campaign, the CTR and conversion rate was too low if compare to the others campaign. This fenomenum should be investigate.

Note: The cost would have a good drop if we stop the announcement to female public. I'd keep announcing for the 40-49 segments because, despite it's more expensive, it had a lot of conversions.

##6.5 Optimization suggestions (conclusion)

###916
 Increase its budget focusing more in Male between 30-39 and interests between 15-38.

###936
Increase its budget, focusing in the age segment 30-39. It could be interesting to keep announcing for Female.

###1178
To lower the cost per conversion, would be interesting to focus more in the 30-39 public. I'd keep the announcing to F public and use the interests that had more results.
Maybe trying to model the ads of campaign_916 and 936 could improve the conversion rates.

###1500
To drop the cost per conversion, should stop the announcement to F public. 
I'd keep announcing for the 40-49 segments because, despite it's more expensive, it had a lot of conversions.

